In [1]:
.libPaths("/home/tbellagio/miniforge3/envs/r-environment/lib/R/library")

In [110]:
library(reticulate)

ERROR: Error in library(reticulate): there is no package called ‘reticulate’


In [2]:
library(data.table)
library(gradientForest)
library(raster)
library(reticulate)

Warning message:
“package ‘data.table’ was built under R version 4.3.3”
Warning message:
“package ‘gradientForest’ was built under R version 4.3.2”
Warning message:
“package ‘raster’ was built under R version 4.3.3”
Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.3.3”


ERROR: Error: package or namespace load failed for ‘raster’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/home/tbellagio/miniforge3/envs/r-environment/lib/R/library/terra/libs/terra.so':
  /home/tbellagio/miniforge3/envs/pipeline_snakemake/lib/R/modules//../../libtiff.so.6: version `LIBTIFF_4.6.1' not found (required by /home/tbellagio/miniforge3/envs/r-environment/lib/R/library/terra/libs/../../../../libgdal.so.36)


In [86]:
# Read the train samples from CSV
train_samples <- fread("train_samples.csv")$train_samples

# Convert the vector of train samples to a list
test_samples <- fread("test_samples.csv")

In [87]:


# Load the Python object using reticulate
#plits_samples <- py_load_object("../leave_1_out/splits_samples.pkl")

# Extract train and test samples
#train_samples <- splits_samples[[1]][[1]]
#test_samples <- splits_samples[[1]][[2]]

# Load the delta_p.csv file using only the train_samples columns
delta_p_file <- '../key_files/merged_hapFIRE_allele_frequency_LDpruned.txt'
delta_p <- fread(delta_p_file, select = train_samples, sep = '\t')


In [88]:
env <- fread('env_train.csv')

In [89]:

# Check the new column names
#colnames(delta_p)

In [90]:
# Calculate maxLevel to account for correlations
maxLevel <- log2(0.368 * nrow(env) / 2) # Adjusted formula for Gradient Forest



In [91]:
maxLevel

[1] 5.852398

In [92]:
# Transpose delta_p to make samples the rows
delta_p_t <- as.data.frame(t(delta_p))

# Check the new dimensions
dim(delta_p_t)  # Should be 314 x 13985

[1]   314 13985

In [93]:
colnames(delta_p_t) <- paste("snp", 1:ncol(delta_p_t), sep = "_")


In [95]:
selected_snps <- sample(colnames(delta_p_t), 1000)

# Subset the delta_p_t DataFrame with the selected SNPs
delta_p_t_subset <- delta_p_t[, selected_snps]

In [97]:
# Combine environmental and allele frequency data
gf_data <- cbind(env, delta_p_t_subset)

In [98]:
dim(delta_p_t) # 13985 * 314

dim(env) # 314 * 19

[1]   314 13985

[1] 314  19

In [106]:
13985/2000

[1] 6.9925

In [107]:
6.9925 * 30

[1] 209.775

In [109]:
209.775 * .5

[1] 104.8875

In [99]:
head(gf_data)

bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,bio10,⋯,snp_11621,snp_5176,snp_13180,snp_8925,snp_11286,snp_6050,snp_5971,snp_3699,snp_10773,snp_3930
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.03608812,0.5466954,0.12848982,0.4866360,0.4728528,0.5921950,0.5639325,0.4882692,0.2202897,0.6143599
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.08682310,0.5549857,0.03293962,0.5590437,0.3016696,0.5766053,0.6520392,0.3505035,0.1817615,0.6382679
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.08879678,0.3494160,0.09556063,0.6468755,0.5148619,0.4265307,0.4948287,0.4075347,0.1794208,0.5154747
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.03871712,0.4558272,0.04546594,0.5378486,0.4817161,0.5058684,0.5550862,0.4553288,0.2473929,0.5659187
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.03107104,0.4271764,0.06496562,0.7183752,0.4911088,0.5702374,0.5598397,0.4460735,0.1901072,0.6032273
15.2511,17.57473,44.37395,690.0336,35.66885,-3.937109,39.60596,11.62251,23.99913,24.0771,⋯,0.01804249,0.4614457,0.02696895,0.5738153,0.5245227,0.6046786,0.3363990,0.4357897,0.3334146,0.4529320


In [102]:
maxLevel <- log2(0.368 * nrow(env) / 2) # Adjusted formula for Gradient Forest

In [103]:
# Measure the time taken to fit the Gradient Forest model
time_taken <- system.time({
  gfRef <- gradientForest(
    data = gf_data, 
    predictor.vars = colnames(env),
    response.vars = colnames(delta_p_t_subset), 
    ntree = 500, 
    maxLevel = maxLevel, 
    trace = TRUE, 
    corr.threshold = 0.50
  )
})

# Print the time taken
print(time_taken)

Calculating forests for 1000 species
...............................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
.........................................................................

In [104]:
804.681/60

[1] 13.41135

In [9]:
head(gf_data)

site,bio1,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,⋯,snp__305,snp__306,snp__307,snp__308,snp__309,snp__310,snp__311,snp__312,snp__313,snp__314
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.0455755,0.0275850,0.008574176,0.1036788,0.1414159,0.0518210,0.02296771,0.02907167,0.00498375,0.0083695
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.4795775,0.3435815,0.598400000,0.7044097,0.6434978,0.4950264,0.75754871,0.70212667,0.69837550,0.7860165
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.3015940,0.0052095,0.081712000,0.1860766,0.1505190,0.1856356,0.04456500,0.16897300,0.00706100,0.1090760
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.5604120,0.4279115,0.517951647,0.5543469,0.8425235,0.5258325,0.74879971,0.44575633,0.57838550,0.7290735
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.5132685,0.3169265,0.236816059,0.5838729,0.4899692,0.6341885,0.17425614,0.62356733,0.41217925,0.3355085
1,10.74391,21.19392,47.68817,716.6599,32.80172,-11.64101,44.44273,2.111033,11.48541,⋯,0.0091610,0.0623930,0.003073235,0.0049760,0.0451350,0.0431150,0.07796929,0.11011533,0.07497500,0.0094380


In [ ]:
# Predict genomic offset for the left-out location
new_env <- env[test_samples, ]
genomic_offset <- predict(gfRef, new_env)

# Display results
print(genomic_offset)

# Save the genomic offset predictions
write.csv(genomic_offset, file = paste0("genomic_offset_", test_samples, ".csv"), row.names = FALSE)

In [ ]:
maxLevel <- log2(0.368*nrow(envGF)/2) #account for correlations, see ?gradientForest 

# Fit gf models for reference SNPs 
gfRef <- gradientForest(cbind(env, delta_p), predictor.vars=colnames(env),
                         response.vars=colnames(delta_p), ntree=500, 
                         maxLevel=maxLevel, trace=T, corr.threshold=0.50)
